# MNIST image classifier

![](https://www.tensorflow.org/images/MNIST.png)

- 손으로 쓴 숫자(0~9) 데이터셋
- Yann Lecun이 만든 딥러닝 연구용 벤치마크 데이터 (http://yann.lecun.com/exdb/mnist/)
- 총 70,000건의 데이터


### Size
- input: 28 x 28 greyscale image
- output: 0~9 numbers

### Representation
![](https://www.tensorflow.org/versions/r0.12/images/MNIST-Matrix.png)
range between 0 and 1, flatten and converted to 1d np array of 784 features

#### references
- https://www.tensorflow.org/get_started/mnist/beginners
- https://www.tensorflow.org/get_started/mnist/pros
- https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/tutorials/mnist/mnist_softmax.py

# load MNIST data

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets('./data', one_hot=True)

print()
print("Image Shape: {}".format(mnist.train.images[0].shape))
print("Label Shape: {}".format(mnist.train.labels[0].shape))
print()
print("Training Set:   {} samples".format(len(mnist.train.images)))
print("Test Set:       {} samples".format(len(mnist.test.images)))

## Visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index = 1
image_sample = mnist.train.images[index].reshape((28, 28))
label_sample = np.argmax(mnist.train.labels[index])
plt.imshow(image_sample, cmap="gray")
plt.title(label_sample)

# The basic model: perceptron

Import tensorflow and set hyperparameters.

In [ ]:
import tensorflow as tf

# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.1

Define placeholders and the perceptron model.

In [ ]:
# define placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
W = tf.Variable(tf.zeros([784, 100]))
b = tf.Variable(tf.zeros([100]))
h = tf.nn.relu(tf.matmul(x, W) + b)

W2 = tf.Variable(tf.zeros([100, 10]))
b2 = tf.Variable(tf.zeros([10]))
y = tf.matmul(h, W2) + b2

Define loss, optimizer, and accuracy.

In [ ]:
# Define loss and optimizer
# cross_entropy = tf.reduce_mean(
#       tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
cross_entropy = tf.reduce_mean(cross_entropy_vector)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Create session and initialize all variables.

In [ ]:
# Create session and initialize all variables (Here, W and b)
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

Train the model.

In [ ]:
# Train
n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

for epoch in range(n_epoch):
    for _ in range(n_batch_per_epoch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('epoch %d: %.4f' % (epoch, test_accuracy))

Don't forget to close the session after training.

In [ ]:
sess.close()

## [참고] MNIST Classification Error Rate (Since 1998~)

CNN을 사용했을 때 0.23% 에러율
- http://yann.lecun.com/exdb/mnist/

the best error rate: 0.21% (ICML 2013)
- http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

### 코드 정리: perceptron

In [ ]:
import tensorflow as tf

# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.1

# define placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Create session and initialize all variables (Here, W and b)
# And then train the model
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

    for epoch in range(n_epoch):
        for _ in range(n_batch_per_epoch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print('epoch %d: %.4f' % (epoch, test_accuracy))

# 실습: More complicated models...

은닉 노드를 늘려보거나 층을 좀 더 쌓아보자.

이 때, `tf.contrib.layers`를 사용해서 코드를 간단화해보자.

### [참고] `tf.contrib.layers`
layer에 대한 high-level wrapper 모음
- https://www.tensorflow.org/api_guides/python/contrib.layers#Higher_level_ops_for_building_neural_network_layers

without tf.contrib.layers
```
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b
```

with tf.contrib.layers
```
x = tf.placeholder(tf.float32, [None, 784])
y = tf.contrib.layers.linear(x, 10)
```

### tf.contrib.layers
`tf.contrib.layers`를 사용하여 아래에 perceptron 모델 코드를 작성하고 파라미터를 학습해보자.

In [ ]:
import tensorflow as tf

# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.1

# define placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
y = tf.contrib.layers.linear(x, 10)

# Define loss and optimizer
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Create session and initialize all variables (Here, W and b)
# And then train the model
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

    for epoch in range(n_epoch):
        for _ in range(n_batch_per_epoch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print('epoch %d: %.4f' % (epoch, test_accuracy))

### Add a hidden layer

input layer와 output layer 사이에 hidden layer를 하나 추가해보자.

In [ ]:
import tensorflow as tf

# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.1

# define placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
h = tf.contrib.layers.fully_connected(x, 100, activation_fn=tf.nn.relu)
y = tf.contrib.layers.linear(h, 10)

# Define loss and optimizer
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Create session and initialize all variables (Here, W and b)
# And then train the model
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

    for epoch in range(n_epoch):
        for _ in range(n_batch_per_epoch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print('epoch %d: %.4f' % (epoch, test_accuracy))

### Going deep: add more than one hidden layer

input layer와 output layer 사이에 hidden layer를 하나 이상 추가해보자.

In [ ]:
import tensorflow as tf

# hyperparameters
n_epoch = 10
batch_size = 100
learning_rate = 0.1

# define placeholders
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# Create the model
h1 = tf.contrib.layers.fully_connected(x, 100, activation_fn=tf.nn.relu)
h2 = tf.contrib.layers.fully_connected(h1, 100, activation_fn=tf.nn.relu)
y = tf.contrib.layers.linear(h2, 10)

# Define loss and optimizer
cross_entropy = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

# Define additional tensors for evaluation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Create session and initialize all variables (Here, W and b)
# And then train the model
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    n_batch_per_epoch = mnist.train.labels.shape[0] // batch_size

    for epoch in range(n_epoch):
        for _ in range(n_batch_per_epoch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
        print('epoch %d: %.4f' % (epoch, test_accuracy))

# SUMMARY

1. Load data
2. Create placeholders for inputs and labels
3. Create weight and bias (It will be unnecessary if you use `tf.contrib.layers`)
4. Inference (`y=f(x)`)
5. Specify loss function (MSE, cross-entropy, ...)
6. Create optimizer
7. Train the model by running optimizer (Don't forget to initialize the variables!)

(option: write log files using a FileWriter to visualize it on TensorBoard!)